<a href="https://colab.research.google.com/github/pakonro/Medic-Project/blob/master/Kopie_von_ipi_data_analysis_2023_session1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# import modules
import cv2, h5py, random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.morphology as morphology
import skimage.measure as measure
from pathlib import Path
from tqdm.auto import tqdm
import shutil, os, requests
from datetime import datetime

In [13]:
# load custom functions
def write_to_h5(h5file, data):
    hf = h5py.File(str(h5file), 'w')
    hf.create_dataset('data', data=data)
    hf.close()

def read_from_h5(h5file):
    hf = h5py.File(h5file, 'r')
    return np.array(hf.get("data"))

def download_file(url, file_name):
    with requests.get(url, stream=True) as r:
        total_length = int(r.headers.get("Content-Length"))
        with tqdm.wrapattr(r.raw, "read", total=total_length, desc=file_name)as raw:
            with open(file_name, 'wb')as output:
                shutil.copyfileobj(raw, output)

data_analysis_results = dict()

# Part 1 | Download Data

In [ ]:
# download data (h5)
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
data_id = 0  # <- change this according to the list below and download the file
             #    you want to download by executing this cell
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------

urls_h5 = [
    ('g1-data-h5', 'https://cloud.tuhh.de/index.php/s/97rR8gW6XZToT3S/download'),  # data_id = 0
    ('g2-data-h5', 'https://cloud.tuhh.de/index.php/s/NegW8RaBdAqa7NW/download'),  # data_id = 1
    ('g3-data-h5', 'https://cloud.tuhh.de/index.php/s/L9dLAgiTe6LGR3k/download'),] # data_id = 2

dir_data_h5 = Path(r'data_h5')
dir_data_h5.mkdir(exist_ok = True)
url = urls_h5[data_id][1]
download_file(url, str(dir_data_h5/"download.zip"))
!unzip data_h5/download.zip -d data_h5
os.remove(dir_data_h5/"download.zip")

data_h5/download.zip:   0%|          | 0/269045196 [00:00<?, ?it/s]

Archive:  data_h5/download.zip
replace data_h5/g1-with_internal_1.1_Umf_1.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Part 2 | Select Data

In [ ]:
# select and load dataset
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
dataset_id = 6   # <- change this to the id of the file you want to load and process
                 #    the dataset_id is related to the list "available datasets"
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------

dataset_list = os.listdir("data_h5")
dataset_list.sort()
print('available datasets:')
for i, dataset in enumerate(dataset_list):
    print("dataset_id="+str(i), dataset)
current_dataset = dataset_list[dataset_id]
print('\ncurrent_dataset is:')
print(current_dataset)
# load dataset
dataset_path = "data_h5/" + current_dataset #comment
video_array = read_from_h5(dataset_path)
video_array = video_array.astype(float) / 255

# Part 3 | Find suitable parameters
This part will help you to determine suitable data processing parameters. Remember: each dataset may require different processing parameters if the camera orientation changed, the crop needs to be adjusted. If the lightning setup changed, the brightness_factor needs to change. The best working threshold for segmentation is connected to the general brightness of the image and therefore changes with the brightness_factor.

### Determine Crop
Further crop the data to the area of interest. Discuss in your group and with the tutors, which areas should be analyzed and which ones should not!

In [ ]:
# -------------------------- change parameters here -------------------------- #
crop_left = 45
crop_right = 50
crop_top = 60
crop_bot = 110
# -------------------------- change parameters here -------------------------- #

random_frame_id = random.randrange(0, video_array.shape[2])
frame = video_array[:,:,random_frame_id]

video_array_cropped = video_array[crop_top:-crop_bot, crop_left:-crop_right, :]
frame_cropped = video_array_cropped[:,:,random_frame_id]

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
axs[0].imshow(frame, cmap='gray',vmin=0, vmax=1)
axs[0].set_title('original image with crop visualization')
axs[0].text(30,50, current_dataset, c="white", size=14, va="top")
rect = patches.Rectangle((crop_left, crop_top), frame.shape[1]-crop_left-crop_right, frame.shape[0]-crop_top-crop_bot, linewidth=1, edgecolor="r", facecolor="none")
axs[0].add_patch(rect)
axs[1].imshow(frame_cropped, cmap='gray',vmin=0, vmax=1)
axs[1].set_title('resulting cropped image')
plt.tight_layout()

### Determine insert masking area

In this section we want to mask the insert. Some part of the fluidized bed is blocked by the insert. We want to exclude this area from the analysis. The excluded area is previewed by the blue rectangle. Adjust the parameters, so that the area matches the actual insert. If you are working with with data without internal, you may switch off masking by setting `masking = False`

Caution: When the crop in the previous cell is changed, these parameters have to be adjusted!

In [ ]:
# -------------------------- change parameters here -------------------------- #
masking = True
pad_left = 0
pad_right = 0
pad_top = 435
pad_bot = 0
# -------------------------- change parameters here -------------------------- #

random_frame_id = random.randrange(0, video_array.shape[2])
frame = video_array_cropped[:,:,random_frame_id]

fig, axs = plt.subplots(1, 1, figsize=(15, 10))
axs.imshow(frame, cmap='gray',vmin=0, vmax=1)
axs.set_title('original image with crop visualization')
axs.text(30,50, current_dataset, c="white", size=14, va="top")
rect = patches.Rectangle((pad_left, pad_top), frame.shape[1]-pad_left-pad_right, frame.shape[0]-pad_top-pad_bot, linewidth=1, edgecolor="b", facecolor="none")
axs.add_patch(rect)
plt.tight_layout()

### Determin Brightness Factor

We want to adjust the global brightness, so that the full value range (0 to 1) is utilized. Make the image as bright as possible, without loosing many details by clipping. Clipping means that values exceed the maximum value 1.0 and therefore get "clipped" back to 1.0. In a heavily clipped / overexposed image, most of the imgage is plain white with many pixel values being exactly 1.0. Use the histogram to find a good value.

If you are not familiar with histograms and how to read them, research this inforamtion.

In [ ]:
# -------------------------- change parameters here -------------------------- #
brightness_factor = 1.6
# -------------------------- change parameters here -------------------------- #

video_array_bright = video_array_cropped * brightness_factor
video_array_bright = np.fmin(video_array_bright, 1.)

random_frame_id = random.randrange(0, video_array_cropped.shape[2])
frame = video_array_cropped[:,:,random_frame_id]
frame_bright = video_array_bright[:,:,random_frame_id]

fig, axs = plt.subplots(1, 3, figsize=(20, 10))
axs[0].imshow(frame, cmap='gray',vmin=0, vmax=1)
axs[0].set_title('original image')
axs[0].text(30,50, current_dataset, c="white", size=14, va="top")
axs[1].imshow(frame_bright, cmap='gray',vmin=0, vmax=1)
axs[1].set_title('cropped image with applied brightness_factor')
axs[2].hist(frame_bright.reshape(-1), bins=128, range=(0,1))
axs[2].set_title('histogram after applied brightness_factor')
plt.tight_layout()

### Determine segmentation threshold

In [ ]:
# plot image grid with various different segmentation thresholds
frame_index = 25
frame = video_array_bright[:,:,frame_index]

fig, axs = plt.subplots(2, 5, figsize=(15,8))
for threshold, ax in zip(np.linspace(start=0.1,stop=0.9,num=10), axs.flatten()):
    computed_mask = frame < threshold
    ax.imshow(computed_mask, cmap='gray', vmin=0, vmax=1)
    ax.set_title("Threshold value = " + str(round(threshold, 2)))
plt.tight_layout()

In [ ]:
# -------------------------- change parameters here -------------------------- #
threshold = 0.7
# -------------------------- change parameters here -------------------------- #

fig, axs = plt.subplots(1, 2, figsize=(10,10))
axs[0].imshow(frame_bright, cmap='gray',vmin=0, vmax=1)
axs[0].set_title("original image")
computed_mask = frame_bright < threshold
axs[1].imshow(computed_mask, cmap='gray')
axs[1].set_title("resulting mask with the threshold you chose")

In [ ]:
# parameter summary
print("These are the parameters you have selected:")
print(f"{crop_left=}, {crop_right=}, {crop_top=}, {crop_bot=}")
print(f"{brightness_factor=}")
print(f"{threshold=}")
print(f"{masking=}")
print(f"{pad_left=}, {pad_right=}, {pad_top=}, {pad_bot=}")
print(f"{threshold=}")

# Part 4 | Segmentation
This script is responsible for the binary mask computation. Familiarize with the filters and functions used.

In [ ]:

from skimage import (
    filters, measure, morphology, segmentation
)
from skimage.data import protein_transport
from scipy import ndimage as ndi
# binary mask computation
# computed_mask1, computed_mask2 etc. are further and furhter refinement steps
frame_index = 25
frame = video_array_bright[:, :, frame_index]


thresh = frame < filters.threshold_local(frame, 191, 'mean')
fill = ndi.binary_fill_holes(thresh)

# step 1: apply threshold
computed_mask1 = thresh# fill#frame < threshold

# step 2: black out the inlet area
computed_mask2 = computed_mask1.copy()
if masking:
    computed_mask2[pad_top:-pad_bot, pad_left:-pad_right] = 0

# step 3: remove small white areas with area <8 pixels
computed_mask3 = morphology.area_opening(computed_mask2, area_threshold=8)

# step 4: dilate (enlarge) white area with disk
disk = morphology.disk(5)
computed_mask4 = morphology.binary_dilation(computed_mask3, selem=disk)

# step 5: remove white areas with area <400 pixels
computed_mask5 = morphology.area_opening(computed_mask4, area_threshold=10**2)

# step 6: remove connected areas that begin at the very top or the very bottom
labels, num = measure.label(computed_mask5, return_num=True)
computed_mask6 = computed_mask5
testpoints = [
    labels[0, int(video_array_bright.shape[1]*0.5)],
    labels[-1, int(video_array_bright.shape[1]*0.25)],
    labels[-1, int(video_array_bright.shape[1]*0.5)],
    labels[-1, int(video_array_bright.shape[1]*0.75)],
]
for testpoint in testpoints:
    if testpoint != 0:
        computed_mask6 = np.where(labels == testpoint, 0, computed_mask6)

# generate figure 2 (mask creation algorithm demonstration, just for your)
fig, axs = plt.subplots(1, 7, figsize=(20,20))
axs[0].imshow(frame, cmap='gray',vmin=0, vmax=1)
axs[0].set_title("test")
axs[1].imshow(computed_mask1, cmap='gray')
axs[0].set_title("computed_mask1")
axs[2].imshow(computed_mask2, cmap='gray')
axs[0].set_title("computed_mask2")
axs[3].imshow(computed_mask3, cmap='gray')
axs[0].set_title("computed_mask3")
axs[4].imshow(computed_mask4, cmap='gray')
axs[0].set_title("computed_mask4")
axs[5].imshow(computed_mask5, cmap='gray')
axs[0].set_title("computed_mask5")
axs[6].imshow(computed_mask6, cmap='gray')
axs[0].set_title("computed_mask6")

# Part 5 | Documentation

Create a documentation for your report. If you run this cell, the figure will be exported as a jpg image. The jpg file also includes the paramters that you have set for processing earlier. Include these documentation images with the settings to your report for documentation! Careful: The images are saved to google colab, remember to download the files to your computer before they are deleted automatically.

In [ ]:
settings_string = "\n".join([
    f"{crop_left=}, {crop_right=}\n{crop_top=}, {crop_bot=}",
    f"{brightness_factor=}",
    f"{threshold=}",
    f"{masking=}",
    f"{pad_left=}, {pad_right=}\n{pad_top=}, {pad_bot=}",
    f"{threshold=}",
])

random_frame_id = random.randrange(0, video_array.shape[2])
frame = video_array[:,:,random_frame_id]
frame_bright = video_array_bright[:,:,random_frame_id]

# generate figure 1 (settings overview for documentation. include this in your report!)
fig, axs = plt.subplots(1, 4, figsize=(25, 12))
axs[0].imshow(frame, cmap='gray', vmin=0, vmax=1)
axs[0].set_title('original image')
axs[0].text(30,50, current_dataset, c="white", size=12, va="top")
axs[0].text(30,120, settings_string, c="white", size=12, va="top")
axs[1].imshow(frame_bright, cmap='gray', vmin=0, vmax=1)
axs[1].set_title('cropped image with applied brightness_factor')
axs[2].imshow(computed_mask6, cmap='gray')
axs[2].set_title('computed binary mask')
axs[3].hist(frame_bright.reshape(-1), bins=128, range=(0,1))
axs[3].set_title('brightness histogram after applied brightness_factor')
# plt.tight_layout()

# save image
timestamp_string = datetime.now().strftime("%Y-%m-%d_%Hh%Mm")
image_filename = f"settings_{current_dataset}_{timestamp_string}.jpg"
plt.savefig(image_filename, dpi=150)